# LangChain Expression Language (LCEL)

This example refers to [LangChain開發手冊(旗標)](https://www.tenlong.com.tw/products/9789863127918)

In [98]:
!pip install langchain langchain_openai rich --quiet

In [99]:
# 匯入套件和金鑰
import os
from google.colab import userdata
from rich import print as pprint

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### 簡單使用 LCEL

In [100]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
chat_model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(
    '{city} 位於那一個國家？')



In [101]:
# LCEL
chain = prompt | chat_model | str_parser

In [102]:
pprint(chain)

RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['city'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['city'],
                    input_types={},
                    partial_variables={},
                    template='{city} 位於那一個國家？'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[
        ChatOpenAI(
            client=<openai.resources.chat.completions.Completions object at 0x7a690bb26140>,
            async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a6918fd9390>,
            root_client=<openai.OpenAI object at 0x7a6919183dc0>,
            root_async_client=<openai.AsyncOpenAI object at 0x7a690bb263e0>,
            model_kwargs={},
            openai_api_key=SecretStr('**********')
        )
    ],
    last=StrOutputParser()
)

In [103]:
print(chain.invoke({"city":"台北"}))

台北市是中華民國（台灣）的首都，位於亞洲東部。


### 手工串接個別物件

In [104]:
content = str_parser.invoke(
    chat_model.invoke(
        prompt.invoke({'city': '台北'})))
print(content)

台北是台灣的首都，它位於中華民國的境內。


In [105]:
class make_chain:
    def __init__(self, runnable_list):
        self.__runnable_list = runnable_list
    def invoke(self, arg):
        for runnable in self.__runnable_list:
            arg = runnable.invoke(arg)
        return arg

find_country_chain = make_chain(
    [prompt, chat_model, str_parser]
)

find_country_chain.invoke({'city': '京都'})

'京都位於日本。'

### 使用 RunnableSequence 類別簡化多層函式的呼叫

In [106]:
from langchain_core.runnables import RunnableSequence

find_country_chain = RunnableSequence(
    prompt,
    chat_model,
    str_parser
)
find_country_chain.invoke({'city': 'New York'})

'New York 位於美國。'

### 使用 LCEL 建立兩個Chain, 有共通參數{city}


In [107]:
find_country_chain = prompt | chat_model | str_parser
find_country_chain.invoke({'city': '巴塞隆納'})

'西班牙。'

In [108]:
lang_template = ChatPromptTemplate.from_template('在{city}講哪一種語言？')
find_lang_chain = lang_template | chat_model | str_parser
find_lang_chain.invoke({'city': '開羅'})

'在開羅，主要講的語言是阿拉伯語，屬於埃及的官方語言。此外，也有少數人口講英語或法語。'

### 使用 RunnableParallel 合併相同參數執行並整合2個 Runnable 物件



In [109]:
from langchain_core.runnables import RunnableParallel
find_country_and_lang_chain = RunnableParallel(
    country=find_country_chain,
    lang=find_lang_chain
)
find_country_and_lang_chain.invoke({'city': '開羅'})

{'country': '開羅位於埃及。',
 'lang': '在開羅，人們主要講阿拉伯語。阿拉伯語是埃及的官方語言，也是最常用的語言。此外，許多居民也會說英語和法語。'}

In [110]:
pprint(find_country_and_lang_chain)

RunnableParallel(
    steps__={
        'country': RunnableSequence(
            first=ChatPromptTemplate(
                input_variables=['city'],
                input_types={},
                partial_variables={},
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['city'],
                            input_types={},
                            partial_variables={},
                            template='{city} 位於那一個國家？'
                        ),
                        additional_kwargs={}
                    )
                ]
            ),
            middle=[
                ChatOpenAI(
                    client=<openai.resources.chat.completions.Completions object at 0x7a690bb26140>,
                    async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a6918fd9390>,
                    root_client=<openai.OpenAI object at 0x7a6919183dc0>,
                    root_async_client=<openai.AsyncOpenAI object at 0x7a690bb263e0>,
                    model_kwargs={},
                    openai_api_key=SecretStr('**********')
                )
            ],
            last=StrOutputParser()
        ),
        'lang': RunnableSequence(
            first=ChatPromptTemplate(
                input_variables=['city'],
                input_types={},
                partial_variables={},
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['city'],
                            input_types={},
                            partial_variables={},
                            template='在{city}講哪一種語言？'
                        ),
                        additional_kwargs={}
                    )
                ]
            ),
            middle=[
                ChatOpenAI(
                    client=<openai.resources.chat.completions.Completions object at 0x7a690bb26140>,
                    async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a6918fd9390>,
                    root_client=<openai.OpenAI object at 0x7a6919183dc0>,
                    root_async_client=<openai.AsyncOpenAI object at 0x7a690bb263e0>,
                    model_kwargs={},
                    openai_api_key=SecretStr('**********')
                )
            ],
            last=StrOutputParser()
        )
    }
)

In [111]:
find_country_and_lang_chain = RunnableParallel({
    'country': find_country_chain,
    'lang': find_lang_chain
})
find_country_and_lang_chain.invoke({'city': '台南'})

{'country': '台南位於台灣。',
 'lang': '在台南主要講的語言是閩南語，此外也常使用華語（中文）。閩南語是台灣本土語言之一，廣泛在台灣南部地區使用，包括台南市。'}

In [112]:
summary_template = ChatPromptTemplate.from_template('{country}{lang}')
summary_chain = (
    {
        'country': find_country_chain,
        'lang': find_lang_chain
    }
    | summary_template)
pprint(summary_chain.invoke({'city': '釜山'}))

ChatPromptValue(
    messages=[
        HumanMessage(
            content='釜山位於南韓。在釜山，主要使用的語言是韓語。然而，由於釜山是韓國的第二大城市，也有許多人會說英
語或其他外語。',
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

錯誤示範

In [113]:
error_chain = {'key': 'hello'} | summary_template

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

## 3-2 LCEL 實用功能

### | 串接可呼叫物件

In [ ]:
from operator import attrgetter
get_messages = attrgetter('messages')
from operator import itemgetter
get_first_item = itemgetter(0)

In [ ]:
summary = (summary_chain
           | get_messages
           | get_first_item
           | str_parser)
summary.invoke({'city': '大阪'})

### 使用 RunnablePassthrough

https://python.langchain.com/v0.1/docs/expression_language/primitives/passthrough/


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

### RunnableBinding

指定額外參數(Ex: bind(stop=[]))

In [ ]:
chain = ({"city": RunnablePassthrough()}
         | prompt.bind()
         | chat_model.bind(stop=["台灣","臺灣"])
         | str_parser)
print(chain.invoke("台北"))

### OpenAI function_calling

In [ ]:
from pydantic import BaseModel, Field
class Search(BaseModel):
    """網路搜尋工具"""
    search_input: str = Field(description="應該要搜尋的關鍵字")

In [ ]:
model = chat_model.bind_tools([Search])
pprint(model.kwargs["tools"])

In [ ]:
prompt = PromptTemplate.from_template("{city}位於哪個國家?")
chain = ({"city": RunnablePassthrough()}
         | prompt
         | model)
pprint(chain.invoke("台北").tool_calls)

In [ ]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
tools_parser = JsonOutputToolsParser()

In [ ]:

chain = ({"city": RunnablePassthrough()}
         | prompt
         | model
         | tools_parser)
pprint(chain.invoke("台北"))

In [ ]:
chat_model.invoke(chain.invoke("台北"))

### 分支與合併

In [114]:
person_template = ChatPromptTemplate.from_template(
    "是誰發明{invention}？")
country_template = ChatPromptTemplate.from_template(
    "{person}來自哪個國家？")

person_chain = ({"invention": RunnablePassthrough()}
              | person_template
              | chat_model
              | str_parser)



In [115]:
person_chain.invoke("珍珠奶茶")

'珍珠奶茶最初是在台灣發明的，由台灣的一家茶館創始人陳三元於1980年代初創造。珍珠奶茶後來在台灣迅速流行開來，並傳播到世界各地。现在，珍珠奶茶已成為全球流行的飲料之一。'

In [116]:
temp_chain = {"person": person_chain} | country_template

In [117]:
temp_chain.invoke("珍珠奶茶")

ChatPromptValue(messages=[HumanMessage(content='珍珠奶茶最初是在臺灣發明的。最早記錄的珍珠奶茶店為1983年開在臺北市光華商場的「珍珠奶茶店」。該飲料的起源可追溯到1980年代，當時一位臺灣小吃攤販在碰巧說話時將豆奶、糖和珍珠混合在一起，意外發現味道十分美味，因此開始製作珍珠奶茶，逐漸在臺灣流行開來。來自哪個國家？', additional_kwargs={}, response_metadata={})])

In [118]:
person_summary_chain = (
    {"person": person_chain}
    | country_template
    | chat_model
    | str_parser
)

person_summary_chain.invoke("珍珠奶茶")

'珍珠奶茶最初被認為是由台灣發明的。'

In [119]:
!pip install grandalf
person_summary_chain.get_graph().print_ascii()

+-----------------------+  
| Parallel<person>Input |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *       

### 根據使用者語氣產生不同回答

This example is adapted from
(https://github.com/iangithub/LangChainLearnBook/blob/main/CH4/demo1/demo1/demo4-7.py)

In [133]:
# 定義情緒分析的提示樣板
sentiment_analysis_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="根據這段話分析情緒，並僅回答 'positive' 或 'negative'：'{user_input}'"
)
# 建立情緒分析的 LLMChain
sentiment_analysis_chain = sentiment_analysis_prompt | chat_model | str_parser

# 負面情緒應對的 PromptTemplate
negative_response_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="使用者說了這段話：'{user_input}'。請給出一段安撫的回應。"
)
negative_response_chain = negative_response_prompt | chat_model | str_parser

# 正面情緒應對的 PromptTemplate
positive_response_prompt = PromptTemplate(
    input_variables=["user_input"],
    template="使用者說了這段話：'{user_input}'。請給出一段正向互動的回應。"
)
positive_response_chain = positive_response_prompt | chat_model | str_parser


def execute_conditional_chain(user_input):
    # 第一步：使用 LLM 來分析情緒
    sentiment_result = sentiment_analysis_chain.invoke({"user_input": user_input})

    # 第二步：根據情緒結果選擇要執行的chain
    if sentiment_result.strip().lower() == "negative":
        # 如果情緒為負面，執行負面應對chain
        return negative_response_chain.invoke({"user_input": user_input})
    else:
        # 如果情緒為正面，執行正面應對鏈結
        return positive_response_chain.invoke({"user_input": user_input})



In [136]:
# 執行 Conditional Chain
execute_conditional_chain("我對於你們的服務感到非常滿意，服務人員很用心，環境也很整潔。")

'謝謝您的肯定和讚美！我們非常高興您對我們的服務感到滿意。我們將繼續努力，提供更好的服務和環境給您，希望您下次再來光臨。感謝您的支持！如果您有任何建議或意見，都歡迎告訴我們，讓我們能更進一步提升服務品質。'

In [135]:
execute_conditional_chain("你的產品怎麼這麼貴? 可以給一些折扣嗎?")

'非常抱歉您覺得我們的產品價格較高。我們堅持使用高品質的材料和生產工藝，確保產品的品質和耐用性。不過我們也會不定期舉辦促銷活動或提供折扣碼，讓顧客可以享有更優惠的價格。請您留下您的聯絡方式，我們會定期通知您我們的優惠活動。感謝您的支持和理解。'

### 零污染計畫書Chain

https://colab.research.google.com/drive/1YTT7-4ezZdhWVK3eJ2fo7qQHQD9r6V1q?usp=sharing

In [123]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
format_instructions = json_parser.get_format_instructions()

In [137]:
chat_model = ChatOpenAI(model='gpt-4o-mini')

In [138]:
# 制定提示模板
prompt1 = ChatPromptTemplate.from_template(
    "請根據{attribute}特性，推薦一種環保的再生能源。請僅提供能源的名稱："
)
prompt2 = ChatPromptTemplate.from_template(
    "在永續發展中，{energy}能源通常用於製造哪種材料？請僅提供能源材料的名稱："
    "{format_instructions}"
)
prompt3 = ChatPromptTemplate.from_template(
    "假設每個國家的能源發展是相等的，哪個國家使用{energy}能源可以做得最好？"
    "請僅提供國家/地區名稱："
)
prompt4 = ChatPromptTemplate.from_template(
    "請結合{material}和{country}，描述一個環境友善的未來生活場景。"
)

prompt2 = prompt2.partial(format_instructions=format_instructions)
# 模型串输出模板
model_parser = chat_model | str_parser

# 能源生成鏈
energy_generator = (
    {"attribute": RunnablePassthrough()}
    | prompt1
    | {"energy": model_parser}
)

# 能源材料
energy_to_material = prompt2 | chat_model | json_parser

# 能源使用做得最好的國家
material_to_country = prompt3 | model_parser

# 结合以上
question_generator = (
    energy_generator
    | {"material": energy_to_material,
       "country": material_to_country}
    | prompt4
)

In [141]:
prompt = question_generator.invoke("零污染")
print(f"最終產生的問題：{prompt.messages[0].content}\n\n"
    "----------------------\n"
    f"AI 回答結果：{chat_model.invoke(prompt).content}")

最終產生的問題：請結合{'energy_material': '氫氣'}和日本，描述一個環境友善的未來生活場景。

----------------------
AI 回答結果：在不遠的未來，日本的城市景觀已經徹底改變，成為一個環境友善、可持續發展的模範。這個場景中，氫氣被廣泛應用，成為日常生活中不可或缺的一部分。

在繁忙的東京街頭，人們騎著電動氫氣摩托車，安靜且無污染，穿梭於高樓大廈之間。街道兩旁，綠色植物爬上建築物的外牆，創造出垂直花園，這些植物不僅美化了環境，還能通過光合作用吸收二氧化碳。

早晨，市民們在社區的氫氣充電站為自己的車輛補充氫燃料，這些站點均由可再生能源供電，如太陽能和風能。充電站旁邊是小型氫氣咖啡館，客人們享用用氫氣加熱的飲品，無需擔心二氧化碳排放。

在這個未來的社會中，房屋都裝備了氫氣燃料電池系統，提供家庭所需的電力與熱能。每個小區都有社區花園和共享農田，居民們利用氫氣供應的能量來進行水耕種植，生產新鮮的有機蔬菜，進一步減少碳足跡。

晚上，城市的燈光由氫燃料電池驅動，柔和而不刺眼，不再有傳統街燈的異味和污染。人們聚集在公園中，享受著清新的空氣和乾淨的環境，孩子們在操場上玩耍，父母們則悠閒地聊天，神情愉悅。

這樣的場景體現了日本在氫氣技術上的前瞻性應用，不僅提升了生活品質，也帶領社會邁向一個更加環保的未來。透過氫氣的普及使用，日本正逐步實現2050年碳中和的目標，為全球可持續發展樹立榜樣。


In [ ]:
!pip install grandalf
question_generator.get_graph().print_ascii()

## 3-3 LCEL 函式應用與分支合併

### RunnableLambda

In [ ]:
from langchain_core.runnables import RunnableLambda

In [ ]:
def commodity(food):
    # 定義每個商店的商品和價格
    items = {
        "熱狗": 50,
        "漢堡": 70,
        "披薩": 100}
    item = items.get(food)
    print(f"{food}價格：{item}")
    return {"price": item}

In [ ]:
food=RunnableLambda(commodity)
food.invoke("披薩")

In [ ]:
prompt = ChatPromptTemplate.from_template("我選擇的商品要多少錢？"
                        "數量{number}價錢{price}")
chain = (
    {
        'price':itemgetter("food") | RunnableLambda(commodity),
        'number':itemgetter("number")
    }
    | prompt
    | chat_model
    | str_parser
)
print(chain.invoke({"food":"漢堡", "number":"101"}))

### RunnableBranch

In [ ]:
chain = (
    PromptTemplate.from_template(
        "根據使用者問題作回答, 將問題分為'要求命令'或是'查詢答案'。\n"
        "<問題>\n{question}\n</問題>\n"
        "分類:"""
    )
    | chat_model
    | str_parser
)

In [ ]:
print(chain.invoke({"question": "立刻使用 Google 搜尋台積電股票"}))
print(chain.invoke({"question": "告訴我什麼是極光"}))

In [ ]:
order_chain = (
    PromptTemplate.from_template(
        "你不會思考只根據命令做回應, 每次回答開頭都以 '是的, 主人' "
        "回覆命令\n"
        "問題: {question}\n"
        "回覆:"
    )
    | chat_model
)
ask_chain = (
    PromptTemplate.from_template(
        "你只能回答知識性相關問題, 任何要求命令不會照做也不會回答,"
        "每次回答開頭都以 '根據我的知識' 回覆命令\n"
        "問題: {question}"
        "回覆:"
    )
    | chat_model
)
defult_chain = (
    PromptTemplate.from_template(
        "請回答問題:\n"
        "問題: {question}\n"
        "回覆:"
    )
    | chat_model
)

### **自訂命令判斷function**

In [ ]:
def route(info):
    if "查詢答案" in info["topic"]:
        return ask_chain
    elif "要求命令" in info["topic"]:
        return order_chain
    else:
        return defult_chain

In [ ]:
from langchain_core.runnables import RunnableLambda

full_chain = ({"topic": chain, "question": lambda x: x["question"]}
             | RunnableLambda(route)
             | str_parser)

In [ ]:
print(full_chain.invoke({"question": "去幫我買東西"}))
print('- '*10)
print(full_chain.invoke({"question": "北極圈是在緯度多少以上？"}))

### 使用 RunnableBranch

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "查詢答案" in x["topic"], ask_chain),
    (lambda x: "要求命令" in x["topic"], order_chain),
    defult_chain,
)

In [ ]:
full_chain = ({"topic": chain, "question": lambda x: x["question"]}
              | branch
              | str_parser)

In [ ]:
print(full_chain.invoke({"question": "幫我寫一篇哈利波特小說短評"}))
print('- '*10)
print(full_chain.invoke({"question": "台北101有多高？"}))